# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [5]:
import json

with open('data/contributors_sample.json', 'r') as f:
    data = json.load(f)

for user in data[:3]:
    print(user)


{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}
{'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}
{'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [8]:
import re

with open('data/contributors_sample.json', 'r') as f:
    data = json.load(f)

domains = set()

for user in data:
    match = re.search(r'@([\w.]+)', user['mail'])
    if match:
        domains.add(match.group(1))

for domain in domains:
    print(domain)

gmail.com
yahoo.com
hotmail.com


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [9]:
def find_user(username):
    with open('data/contributors_sample.json', 'r') as f:
        data = json.load(f)

    for user in data:
        if user['username'] == username:
            return user

    raise ValueError('User with username "{}" not found'.format(username))

user = find_user('vickitaylor')
print(user)

{'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [11]:
with open('data/contributors_sample.json', 'r') as f:
    data = json.load(f)

male_count = 0
female_count = 0

for user in data:
    if user['sex'] == 'M':
        male_count += 1
    elif user['sex'] == 'F':
        female_count += 1

print('Number of males:', male_count)
print('Number of females:', female_count)

Number of males: 2064
Number of females: 2136


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [12]:
import pandas as pd

with open('data/contributors_sample.json', 'r') as f:
    data = json.load(f)

contributors = pd.DataFrame(data, columns=['id', 'username', 'sex'])

print(contributors.head())

        id     username sex
0    35193      uhebert   F
1    91970  vickitaylor   F
2  1848091  sheilaadams   F
3    50969     nicole82   F
4   676820       jean67   M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [55]:
contributors = pd.read_json('data/contributors_sample.json')
recipes = pd.read_csv('data/recipes_sample.csv')

merged = pd.merge(recipes, contributors, left_on='contributor_id', right_on='id', how='left')

missing_count = merged[merged['username'].isnull()]['contributor_id'].nunique()
print('Number of missing contributors:', missing_count)

Number of missing contributors: 4204


### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [28]:
import pickle

positions = {}

for user in data:
    for position in user['jobs']:
        if position not in positions:
            positions[position] = [user['username']]
        else:
            positions[position].append(user['username'])

with open('positions.pickle', 'wb') as f:
    pickle.dump(positions, f)

for job, usernames in positions.items():
    print(job, usernames[:3])

Energy engineer ['uhebert', 'annmoore', 'garysilva']
Engineer, site ['uhebert', 'nancy12', 'andrea03']
Environmental health practitioner ['uhebert', 'jonathanchristian', 'xjohnson']
Biomedical scientist ['uhebert', 'smithheather', 'epittman']
Jewellery designer ['uhebert', 'lopezantonio', 'ojames']
Music therapist ['vickitaylor', 'michael28', 'susan16']
Volunteer coordinator ['vickitaylor', 'areyes', 'kevinmckenzie']
Designer, interior/spatial ['vickitaylor', 'joseph54', 'mendozarobert']
Management consultant ['sheilaadams', 'robertrichardson', 'moraleshayley']
Engineer, structural ['sheilaadams', 'karenhuang', 'staceynguyen']
Lecturer, higher education ['sheilaadams', 'woodmarissa', 'kjenkins']
Theatre manager ['sheilaadams', 'icervantes', 'chase81']
Designer, textile ['sheilaadams', 'kaylanichols', 'gmiller']
Mechanical engineer ['nicole82', 'sierraterrell', 'sextonelizabeth']
Retail banker ['nicole82', 'jonathan18', 'toconnor']
Barrister ['nicole82', 'adamsmith', 'joseph98']
Network

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [29]:
jobs_dict = {}

for user in data:
    for job in user['jobs']:
        if job not in jobs_dict:
            jobs_dict[job] = []
        jobs_dict[job].append(user['username'])

# Сохраняем в формате pickle
with open('job_people.pickle', 'wb') as f:
    pickle.dump(jobs_dict, f)

# Сохраняем в формате JSON
with open('job_people.json', 'w') as f:
    json.dump(jobs_dict, f, indent=4)

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [31]:
with open('job_people.pickle', 'rb') as f:
    jobs_dict = pickle.load(f)

for job, usernames in jobs_dict.items():
    print(job, usernames[:3])

Energy engineer ['uhebert', 'annmoore', 'garysilva']
Engineer, site ['uhebert', 'nancy12', 'andrea03']
Environmental health practitioner ['uhebert', 'jonathanchristian', 'xjohnson']
Biomedical scientist ['uhebert', 'smithheather', 'epittman']
Jewellery designer ['uhebert', 'lopezantonio', 'ojames']
Music therapist ['vickitaylor', 'michael28', 'susan16']
Volunteer coordinator ['vickitaylor', 'areyes', 'kevinmckenzie']
Designer, interior/spatial ['vickitaylor', 'joseph54', 'mendozarobert']
Management consultant ['sheilaadams', 'robertrichardson', 'moraleshayley']
Engineer, structural ['sheilaadams', 'karenhuang', 'staceynguyen']
Lecturer, higher education ['sheilaadams', 'woodmarissa', 'kjenkins']
Theatre manager ['sheilaadams', 'icervantes', 'chase81']
Designer, textile ['sheilaadams', 'kaylanichols', 'gmiller']
Mechanical engineer ['nicole82', 'sierraterrell', 'sextonelizabeth']
Retail banker ['nicole82', 'jonathan18', 'toconnor']
Barrister ['nicole82', 'adamsmith', 'joseph98']
Network

Если файл job_people.pickle был записан корректно в предыдущем задании, этот код должен вывести содержимое словаря jobs_dict, который был создан на основе файла contributors_sample.json и содержит информацию о том, какие люди занимали какие должности.

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [32]:
import xml.etree.ElementTree as ET

tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

steps_dict = {}

for recipe in root.findall('recipe'):
    recipe_id = recipe.get('id')
    steps = [step.text.strip() for step in recipe.findall('step')]
    steps_dict[recipe_id] = steps

with open('steps_sample.json', 'w') as f:
    json.dump(steps_dict, f)

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [47]:
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

steps_dict = {}

for recipe in root.findall('recipe'):
    id_prescription = recipe.find('id').text
    steps = recipe.find('steps').findall('step')
    num_steps = len(steps)
    if num_steps not in steps_dict:
        steps_dict[num_steps] = [id_prescription]
    else:
        steps_dict[num_steps].append(id_prescription)

for step, id in steps_dict.items():
    print(step, id[:3])

11 ['44123', '302399', '375376']
3 ['67664', '118843', '147477']
5 ['38798', '69190', '125195']
7 ['35173', '109818', '66932']
4 ['84797', '95926', '134085']
6 ['44045', '306168', '325714']
8 ['107229', '463219', '39172']
12 ['453467', '226001', '129919']
15 ['50662', '180817', '133326']
10 ['503475', '149593', '95534']
18 ['200148', '137701', '122791']
38 ['310570', '392181', '51546']
13 ['141939', '303944', '299968']
14 ['250883', '120297', '296983']
9 ['342620', '342619', '287778']
33 ['129581', '111422', '178749']
2 ['116741', '44050', '292568']
30 ['487173', '319600', '479954']
24 ['447429', '170081', '490199']
28 ['173730', '336627', '214541']
34 ['390230', '70198', '460866']
16 ['193719', '459738', '489337']
29 ['435816', '450975', '204012']
22 ['198073', '387435', '482020']
20 ['290187', '255985', '400025']
1 ['33246', '125721', '8827']
26 ['478546', '429949', '277764']
19 ['449768', '128544', '423528']
17 ['64631', '192212', '29512']
25 ['104588', '26809', '146096']
45 ['32557

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [50]:
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

recipes_with_time = []

for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id').text
    steps = recipe.findall('steps/step')
    for step in steps:
        if 'has_minutes' in step.attrib or 'has_hours' in step.attrib:
            recipes_with_time.append(recipe_id)
            break

print(recipes_with_time[:30])

['44123', '35173', '453467', '306168', '50662', '118843', '149593', '200148', '310570', '95534', '109818', '66932', '226001', '141939', '250883', '120297', '223349', '60938', '302399', '342620', '296983', '129581', '325714', '487173', '447429', '137701', '63346', '342619', '383120', '463219']


3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [65]:
import csv

# Load data from CSV file
recipes = pd.read_csv('data/recipes_sample.csv')

# Load data from XML file
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

# Create dictionary of steps for each recipe
steps_dict = {}
for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id').text
    steps = recipe.findall('steps/step')
    steps_dict[recipe_id] = len(steps)

# Fill missing n_steps values
for i, row in recipes.iterrows():
    recipe_id = row['id']
    n_steps = row['n_steps']
    if pd.isna(n_steps):
        if recipe_id in steps_dict:
            recipes.at[i, 'n_steps'] = steps_dict[recipe_id]

# Display the updated DataFrame
print(recipes)

                                               name      id  minutes  \
0             george s at the cove  black bean soup   44123       90   
1                healthy for them  yogurt popsicles   67664       10   
2                      i can t believe it s spinach   38798       30   
3                              italian  gut busters   35173       45   
4          love is in the air  beef fondue   sauces   84797       25   
...                                             ...     ...      ...   
29995  zurie s holey rustic olive and cheddar bread  267661       80   
29996          zwetschgenkuchen  bavarian plum cake  386977      240   
29997   zwiebelkuchen   southwest german onion cake  103312       75   
29998                                   zydeco soup  486161       60   
29999        cookies by design   cookies on a stick  298512       29   

       contributor_id   submitted  n_steps  \
0               35193  2002-10-25      NaN   
1               91970  2003-07-26      NaN 

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [66]:
# Load recipes data
recipes = pd.read_csv('data/recipes_sample.csv')

# Check for missing values
if recipes['n_steps'].isnull().any():
    print('Column "n_steps" contains missing values')
else:
    # Convert n_steps to integer type
    recipes['n_steps'] = recipes['n_steps'].astype(int)
    # Save to file
    recipes.to_csv('recipes_sample_with_filled_nsteps.csv', index=False)

print(recipes)

Column "n_steps" contains missing values
                                               name      id  minutes  \
0             george s at the cove  black bean soup   44123       90   
1                healthy for them  yogurt popsicles   67664       10   
2                      i can t believe it s spinach   38798       30   
3                              italian  gut busters   35173       45   
4          love is in the air  beef fondue   sauces   84797       25   
...                                             ...     ...      ...   
29995  zurie s holey rustic olive and cheddar bread  267661       80   
29996          zwetschgenkuchen  bavarian plum cake  386977      240   
29997   zwiebelkuchen   southwest german onion cake  103312       75   
29998                                   zydeco soup  486161       60   
29999        cookies by design   cookies on a stick  298512       29   

       contributor_id   submitted  n_steps  \
0               35193  2002-10-25      NaN   
1 